In [1]:
from pyvis.network import Network
import pandas as pd
from collections import defaultdict

In [2]:
df = pd.read_csv("triplets.csv")

# 初始化 pyvis 图
net = Network(height="800px", width="100%", bgcolor="#222222", font_color="white", notebook=True)
net.toggle_physics(True)

# 统计节点频率
node_freq = defaultdict(int)
node_type = defaultdict(lambda: "Unknown")  # 论文/会议/关键词
for _, row in df.iterrows():
    h, r, t = row['head'], row['relation'], row['tail']
    node_freq[h] += 1
    node_freq[t] += 1

    if r == "published_in":
        node_type[h] = "Paper"
        node_type[t] = "Conference"
    elif r == "has_keyword":
        node_type[h] = "Paper"
        node_type[t] = "Keyword"
    elif r == "belongs_to":
        node_type[h] = "Keyword"
        node_type[t] = "Topic"

# 会议专属颜色
conference_colors = {
    "AAAI": "#FF4136",     # 红色
    "CVPR": "#0074D9",     # 蓝色
    "ICML": "#2ECC40",     # 绿色
    "KDD": "#B10DC9",      # 紫色
    "NeurIPS": "#FF851B"   # 橙色
}

default_color_map = {
    "Paper": "#7FDBFF",     # 浅蓝
    "Conference": "#AAAAAA",
    "Keyword": "#01FF70",   # 亮绿
    "Topic": "#F012BE",     # 粉紫
    "Unknown": "#DDDDDD"
}


In [3]:
# 添加控件：初始化图
net = Network(height="800px", width="100%", bgcolor="#222222", font_color="white", notebook=True)
net.toggle_physics(True)
net.show_buttons(filter_=['physics'])  # 可调节引力模型

In [4]:
def normalize_size(freq, min_size=10, max_size=40):
    min_freq = min(node_freq.values())
    max_freq = max(node_freq.values())
    if max_freq == min_freq:
        return (min_size + max_size) / 2
    return min_size + (freq - min_freq) / (max_freq - min_freq) * (max_size - min_size)

In [5]:
added_nodes = set()
paper_conference_map = {}  

for _, row in df.iterrows():
    h, r, t = row['head'], row['relation'], row['tail']

    # 记录论文 -> 会议 映射
    if r == "published_in":
        paper_conference_map[h] = t

    for node in [h, t]:
        if node not in added_nodes:
            size = normalize_size(node_freq[node])

            if node in conference_colors:  # 是会议本身
                color = conference_colors[node]
            elif node in paper_conference_map:  # 是论文，根据会议着色
                conf = paper_conference_map.get(node)
                color = conference_colors.get(conf, default_color_map["Paper"])
            else:
                ntype = node_type.get(node, "Unknown")
                color = default_color_map.get(ntype, "gray")

            net.add_node(node, label=node, title=f"{node_type[node]}: {node}", size=size, color=color)
            added_nodes.add(node)

    net.add_edge(h, t, label=r)

In [6]:
net.show("knowledge_graph.html")  # 输出为 HTML 文件
print("已生成")

knowledge_graph.html
已生成
